In [5]:
from bs4 import BeautifulSoup

with open('./nba-standing-20211128.htm', 'r', encoding='utf-8') as f:
    html = f.read()

soup = BeautifulSoup(html, 'html.parser')

In [7]:
td_list = soup.find_all('td', 'liveresults-sports-immersive__match-tile')
len(td_list)

421

In [8]:
table_list = soup.find_all('table', 'KAIX8d')
len(table_list)

421

In [9]:
table = table_list[0]
tr_list = table.find_all('tr', 'L5Kkcd')
len(tr_list)

2

In [10]:
tr = tr_list[0]
td_list = tr.find_all('td', 'tns-c')

len(td_list)

1

In [11]:
td = td_list[0]
td

<td class="tns-c imspo_mt__tt-w imspo_mt__dt-t"><div class="imspo_mt__t-sc"><div class="imspo_mt__tt-w">123</div></div><div class="ellipsisize" data-df-team-mid="/m/0jm3b"><span>Nets</span></div></td>

In [17]:
td.find_all('div', 'imspo_mt__tt-w')[0].get_text()

'123'

In [18]:
td.find_all('div', 'ellipsisize')[0].get_text()

'Nets'

In [22]:
tr = tr_list[1]

td_list = tr.find_all('td', 'tns-c')
td = td_list[0]

td.find_all('div', 'imspo_mt__tt-w')[0].get_text()
td.find_all('div', 'ellipsisize')[0].get_text()

'Lakers'

In [44]:
def get_scores(table):
    def get_score(tr):
        td_list = tr.find_all('td', 'tns-c')
        if len(td_list) < 1:
            raise ValueError(f'There is no <td> element.')
            
        td = td_list[0]
        
        div_list = td.find_all('div', 'imspo_mt__tt-w')
        if len(div_list) < 1:
            raise ValueError(f'There is no scoring <div> element.')
            
        score = div_list[0].get_text()
        
        div_list = td.find_all('div', 'ellipsisize')
        if len(div_list) < 1:
            raise ValueError(f'There is no team <div> element.')
        
        team = div_list[0].get_text()
        
        return team, int(score)
    
    def is_preseason():
        div_list = table.find_all('div', 'imspo_mt__lg-st-co')
        if len(div_list) > 0:
            return True
        return False
    
    try:
        tr_list = table.find_all('tr', 'L5Kkcd')
        if len(tr_list) != 2:
            raise ValueError(f'Table does not have 2 rows; table has {len(tr_list)} rows.')

        a_team, a_score = get_score(tr_list[0])
        h_team, h_score = get_score(tr_list[1])
        
        return {
            'a_team': a_team,
            'h_team': h_team,
            'a_score': a_score,
            'h_score': h_score,
            'preseason': is_preseason()
        }
    except:
        return None

get_scores(table_list[0])

{'a_team': 'Nets',
 'h_team': 'Lakers',
 'a_score': 123,
 'h_score': 97,
 'preseason': True}

In [50]:
import pandas as pd

scores = (get_scores(t) for t in table_list)
scores = filter(lambda s: s is not None, scores)

df = pd.DataFrame(scores)
df.shape

(365, 5)

In [51]:
df.head()

,a_team,h_team,a_score,h_score,preseason
0,Nets,Lakers,123,97,True
1,76ers,Raptors,107,123,True
2,Hawks,Heat,99,125,True
3,Magic,Celtics,97,98,True
4,Pelicans,Timberwolves,114,117,True


In [52]:
df.tail()

,a_team,h_team,a_score,h_score,preseason
360,Magic,Cavaliers,92,105,False
361,Heat,Bulls,107,104,False
362,Hornets,Rockets,143,146,False
363,Wizards,Mavericks,120,114,False
364,Pelicans,Jazz,105,127,False


In [53]:
df.to_csv('2021.csv', index=False)